In [1]:
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite

import eli5
#import lime
#from lime.lime_text import LimeTextExplainer


from sklearn_crfsuite import CRF 
from sklearn.model_selection import cross_val_predict # train_test_split <- 쓰려면 쓰기
from sklearn_crfsuite .metrics import flat_classification_report # metrics: measure(Accuracy, score 등) 도구

from nltk.probability import FreqDist
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma as kkm
from konlpy.tag import Okt
#from konlpy.tag import Mecab
#import Mecab
import tweepy


# score 정보 -> (원하는 것) 어떤 문장이 몇점인지를 예측 -> 이 값이 평균 이상이면 긍정, 평균 미만이면 부정
#            -> 긍/부정 으로만 학습하면 결과가 더 부정확할 수도. 
#            -> (현행) 평균 이상이면 100, 미만 0으로 분류해서 학습, 적용

# kkm 형태소 분석기에서 사전 조정해서, 칼퇴, 워라밸 같은 우리가 사용하는 단어로 수정하기.
# raw data : 맞춤법 교정.
# 조사 같은 연결어 제외하기. feature 추가(전전단어와 후후단어)
# CRF : N-gram ?????  kkm 사전을 형태소로 나눠놓고 하는 걸로. 

# 긍정/부정 리뷰 스크래핑+키워드 추출+사전으로 추가(feature) 
# 명사(를 비롯한 주요 단어)인 pos tag 만 뽑아서 feature에서 가중치 주기. 
# feature: pos tag에서 규칙 찾아서(eg. NNG+NNG) 여러 단어 연접하여 새롭게 긍/부정어 사전 돌리기 ??? <- 복잡. 
# 쪼개진 긍부정어 사전(만들어진 txt) 찾아보기.

# 조사 없애기? pos tag 포기. 띄어쓰기 교정 + 맞춤법 교정 + CRF ...? 
# feat -1, +1 < 최고로 나쁘다 -> 최고 로 나쁘 다 : 이런 식으로 CRF 특징이 무효화 됨.  

# CRF가 아니라, 다른 방식으로 해보는 게 어떤지? eg. review classification처럼 
# summarization + 빈도수 처리 + .... + N-gram 활용
# 시각화: eli5 

In [2]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [3]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [4]:
#dict_pos=[]
#dict_neg=[]
#for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/pos_pol_word.txt", 'rt', encoding='UTF8'):
#    dict_pos.append(line.split('\n')[0])
    
#for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/neg_pol_word.txt", 'rt', encoding='UTF8'):
#    dict_neg.append(line.split('\n')[0])

In [5]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1=dict(ind_reviews(1001))

In [6]:
reviews1

{'계약직은  아무 생각없이 다니기 좋은곳. 정규직으로 다니기엔 다소 피곤할수도': '66',
 '필요한 부분 잘 지원해주고 정해진 일만 하면 자유시간이 보장됩니다. 아르바이트라서 편하게 일했어요.': '66',
 '일이 꽤 많음. 세 달 안에 소화하면 매우 많은 도움이 될 것 같으나, 소진되지 않도록 스스로 관리를 잘 하는 것이 중요함. 행정원분들과 연구원분들의 사이가 좋은 편이어서 사람들로 인한 스트레스는 덜 받으면서 일에 집중할 수 있음.': '66',
 '직원들과의 사이가 돈득하고 기업 분위기가 좋은면도 있음': '66',
 '이름이 유명한 만큼 좋은 기업. 공기업인만큼 편하다.자기계발을 할 수 있는 시간이 주어진다고 할 수 있을 만큼 여가시간이 많다.': '66',
 '정부기관답게 조직이 크고 체계적이었습니다.': '66',
 '맡은 분야의 전문가가 될 수 있는 곳. 분위기가 딱딱하지 않음.': '66',
 '다양한 자기개발을 할 수 있고 누구나 쉽게 이용 할 수 있음': '66',
 '이름만 들으면 도박장인 듯하여 꺼려졌으나 다른 서비스직이랑 비슷한 업무를 진행함': '64',
 '개인에게 주어진 일만 다하면 터치가 없는 회사.사람 스트레스 절대없음. 업무량은 공공기관 치고 많은편.정년보장.이직 어려움.심도있는 전문지식 습득 어려움.': '64',
 '계약직은 짐짝 취급당함 단물 빼먹고 버리기 잘함 자유로운 휴가사용 가능': '78',
 '좋은 사람들과 보람차게 일할 수 있는 직장': '78',
 '대체로 여유로운 분위기 그러나 사람마다 업무 강도 차이가 커보얐음': '78',
 '넓은 캠퍼스에서 일할 수 있고 대학교 시설을 자유롭게 이용할 수 있음. 대신, 다른 지자체에 비해 체계가 없는 편이라고 함.': '78',
 '워라벨이 잘지켜지고, 기업문화가 상당히 수평적이라고  할 수 있다.': '78',
 '보람차게 일할 수 있는 곳. 지식을 얻고자 오는 분들께 서비스를 제공할 수 있다는게 보람찼음.': '78',
 '그냥 저냥 공무윈입니다 다른 자치

In [7]:
text1=list(reviews1.keys())
score1=list(reviews1.values())

In [8]:
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  64.447


In [9]:
#############긍부정분석기 추가!!!!
#############
#############
#############


In [10]:
#전체 리뷰 추출

def all_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    reviews = {}
    for i in range(1, pages+1):
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [11]:
#전체 리뷰 및 별점 딕셔너리화
codes = ['1001', '900', '704', '1004']
reviews={}

for i in range(0,len(codes)):
    reviews.update(dict(all_reviews(codes[i])))

In [12]:
len(reviews)

2547

In [13]:
text_all=list(reviews.keys())
score_all=list(reviews.values())

In [14]:
#긍/부정 사전을 만들기 위한 전체 평균값 산출
sum=0
for i in range(len(score1)):
    sum+=int(score_all[i])
avr_all=sum/len(score_all)
print("score 평균값: ", round(avr_all,3))

score 평균값:  9.06


In [15]:
#공통/ 업종별 불용어(자주 나오지만 문서 전체에 관련되어 있어 성능 체크에 의미가 없는 단어들) 선언
stopwords=["직장","회사","기업","업무","사람","직원","있음","근무","분위기"]
stopwords1=["공공기관","공기업","공공","정부","부처","공무원","기관"]
stopwords2=["은행","금융","금융권","금융업","운용사","보험사","지점"]
stopwords3=["개발자","소프트웨어","IT"]
stopwords4=["세무","회계","회계사","세무사","지점","법인"]

In [16]:
#긍/부정 리뷰별 빈출단어 추출 및 사전화
#검색 성능을 높이기 위해 문장을 제일 잘게 쪼개는 kkma 사용

kkm=kkm()

text_kkm=[]

#형태소 분석 및 문장별 점수 매칭

for i in range(len(text1)):
    text_kkm.append(kkm.nouns(text_all[i]))
    text_kkm[i].append(score_all[i])
    
print(text_kkm)

[['대체적', '근무', '환경', '부서', '업무량', '시', '야근', '필요', '경우', '존재', '100'], ['정년', '보장', '도서', '도서대출반납', '대출', '반납', '다양', '프로그램', '운영', '운영중', '중', '100'], ['대한', '대한민국', '민국', '식량', '식량산업', '산업', '최전선', '분투', '해군', '소수', '소수정예', '정예', '종자', '종자관리', '관리', '전문', '전문기관', '기관', '소속', '소속직원', '직원', '대한민국전체', '전체', '200', '200명', '명', '96'], ['수평적', '회사', '회사분위기', '분위기', '상사', '눈치', '안', '안보고', '보고', '휴가', '내가', '일', '외국인', '사람', '저녁', '삶', '가능', '직장', '96'], ['민원인', '업무', '업무강도', '강도', '조사관', '친절', '94'], ['세종', '세종시', '시', '활동', '기업', '기업지원기관', '지원', '기관', '신생', '신생기관', '94'], ['도청', '근무', '충청남', '충청남도', '도', '총괄적', '업무', '수행', '예하', '시', '시군의', '군의', '정책', '그림', '곳', '94'], ['서울', '도심', '국제기구', '국제', '국제현안', '현안', '수', '관련', '커리어', '있음', '94'], ['자유', '분위기', '보수적', '공공', '공공기관', '기관', '생각', '때', '느낌', '유사', '94'], ['노동법', '곳', '직원', '전반적', '함', '악성', '민', '민원인', '원인', '상대', '가나', '있음', '94'], ['워라벨', '연봉', '세금', '해외', '지역', '근무', '점', '단점', '힘', '92'], ['근무', '연가', '보', '보수가', '수가', '일', '만족

In [17]:
pos_kkm=[]
morphs_kkm=[]
dic_stopwords=stopwords1+stopwords2+stopwords3+stopwords4+stopwords
#연관이 낮아 보이는 형태소와 1글자 음절, 불용어를 제외한 체언, 용언, 관형사, 부사만 뽑아내기

for i in range(len(text_all)):
    pos_kkm.append(kkm.pos(text_all[i]))

dic_kkm=[]

for i in range(len(pos_kkm)):
    dic_kkm.append([])
    for j in range(len(pos_kkm[i])):
        if pos_kkm[i][j][0] not in dic_stopwords:
            if pos_kkm[i][j][1].startswith('N') or pos_kkm[i][j][1].startswith('V')or pos_kkm[i][j][1].startswith('M'):
                dic_kkm[i].append(pos_kkm[i][j][0])
    dic_kkm[i].append(int(score_all[i]))


In [18]:
dic_kkm

[['대체적',
  '환경',
  '좋',
  '부서',
  '따르',
  '다르',
  '업무량',
  '몰리',
  '시',
  '야근',
  '필요',
  '경우',
  '존재',
  100],
 ['정년',
  '확실히',
  '보장',
  '도서',
  '대출',
  '반납',
  '아니',
  '다양',
  '프로그램',
  '운영',
  '중',
  '이',
  '좋',
  100],
 ['대한민국',
  '식량',
  '산업',
  '최전선',
  '분투',
  '해군',
  '같',
  '소수',
  '정예',
  '종자',
  '관리',
  '전문',
  '소속',
  '대한민국',
  '전체',
  '200',
  '명',
  '없',
  96],
 ['수평적',
  '이',
  '좋',
  '상사',
  '눈치',
  '많이',
  '안',
  '보고',
  '휴가',
  '쓰',
  '내가',
  '하',
  '일',
  '열심히',
  '하',
  '되',
  '외국인',
  '일하',
  '것',
  '좋아하',
  '즐기',
  '저녁',
  '있',
  '삶',
  '가능',
  96],
 ['민원인', '많', '강도', '높', '조사관', '친절', 94],
 ['세종', '시', '활동', '좋', '지원', '이', '신생', '이', 94],
 ['도청',
  '충청남도',
  '관하',
  '총괄적',
  '이',
  '수행',
  '되',
  '예하',
  '시',
  '군의',
  '정책',
  '크',
  '그림',
  '그리',
  '곳',
  94],
 ['서울',
  '도심',
  '있',
  '국제기구',
  '국제',
  '현안',
  '다루',
  '수',
  '있',
  '관련',
  '커리어',
  '쌓',
  '수',
  94],
 ['자유', '보수적', '이', '않', '상당히', '생각', '때', '느낌', '가장', '유사', 94],
 ['노동법',
  '제대로',
  '지키',
  

In [19]:
# 긍/부정 빈도수 체크하여 상위 50개를 사전으로 저장

dic_text_pos=[]
dic_text_neg=[]  

for i in range(len(dic_kkm)):
    for j in range(0, len(dic_kkm[i])):
        if type(dic_kkm[i][j]) != int:
            if len(dic_kkm[i][j])>1:
                if float(dic_kkm[i][ -1]) > avr:
                    dic_text_pos.append(dic_kkm[i][j])
                else:
                    dic_text_neg.append(dic_kkm[i][j])

fdist1=FreqDist(dic_text_pos)
dic_text_pos=fdist1.most_common(100)

fdist1=FreqDist(dic_text_neg)
dic_text_neg=fdist1.most_common(100)

In [20]:
dic_text_pos1=[]
dic_text_neg1=[]

for i in range(len(dic_text_pos)):
    dic_text_pos1.append(dic_text_pos[i][0])
    
for i in range(len(dic_text_neg)):
    dic_text_neg1.append(dic_text_neg[i][0])

print(dic_text_pos1)
print(dic_text_neg1)

['일하', '경험', '다니', '다양', '문화', '배우', '환경', '복지', '생각', '가지', '성장', '편하', '안정적', '연봉', '최고', '조직', '계약', '추천', '아니', '가능', '자유', '많이', '나쁘', '시간', '대하', '발전', '워라밸', '급여', '매우', '보장', '강도', '대우', '업계', '괜찮', '수평적', '여러', '워라벨', '투자', '생활', '힘들', '함께', '커리어', '위하', '사업', '본인', '너무', '노력', '따르', '자기', '기회', '비하', '사회', '경력', '정도', '체계', '자부심', '느끼', '장점', '인턴', '부서', '성과', '개발', '빠르', '스타트', '외국계', '지원', '없이', '다만', '그냥', '보이', '하나', '안정성', '수준', '운용', '다르', '관련', '보수적', '전반적', '하지만', '한국', '신입', '능력', '때문', '만큼', '모두', '업무량', '야근', '느낌', '만족', '그렇', '특성', '위치', '출퇴근', '이상', '부족', '월급', '다른', '편안', '정규직', '눈치']
['다니', '일하', '배우', '생각', '추천', '문화', '다양', '경험', '복지', '연봉', '아니', '대하', '많이', '힘들', '경력', '너무', '성장', '환경', '급여', '괜찮', '신입', '편하', '발전', '자유', '가지', '체계', '가능', '그냥', '개인', '사무실', '안정적', '따르', '나쁘', '매우', '시간', '하지만', '사업', '다르', '대표', '대우', '위하', '계약', '업계', '못하', '보수적', '모르', '없이', '이상', '본인', '개발', '운영', '조직', '야근', '보험', '바뀌', '오래', '입사', '자기', '사내', '능력', '워라밸', '강도', '다른', 

In [21]:
import site; site.getsitepackages()


['C:\\ProgramData\\Anaconda3\\envs\\tm',
 'C:\\ProgramData\\Anaconda3\\envs\\tm\\lib\\site-packages']

In [22]:
sent_num=0
text1_sent=[]
#mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
for i in range(len(text1)):
    text1_pos=kkm.pos(text1[i])
    sent_num+=1
    for t in text1_pos:
        n=list(t)
        
        sent_number="Sentence: {}".format(sent_num)
        n.insert(0, sent_number)
        '''
        if n[1] in dict_pos: 
            pos_dict=True
        else: 
            pos_dict=False
        n.insert(3, pos_dict)
        if n[1] in dict_neg: 
            neg_dict=True
        else: 
            neg_dict=False
        n.insert(4, neg_dict)
        '''
        if float(score1[(sent_num-1)]) > avr:
            n.insert(3, '100')
        else: 
            n.insert(3, '0')
        
        #n.insert(5, score1[(sent_num-1)])
        
        text1_sent.append(n)
        

pprint.pprint(text1_sent)

[['Sentence: 1', '계약', 'NNG', '100'],
 ['Sentence: 1', '직', 'NNG', '100'],
 ['Sentence: 1', '은', 'JX', '100'],
 ['Sentence: 1', '아무', 'MDT', '100'],
 ['Sentence: 1', '생각', 'NNG', '100'],
 ['Sentence: 1', '없이', 'MAG', '100'],
 ['Sentence: 1', '다니', 'VV', '100'],
 ['Sentence: 1', '기', 'ETN', '100'],
 ['Sentence: 1', '좋', 'VA', '100'],
 ['Sentence: 1', '은', 'ETD', '100'],
 ['Sentence: 1', '곳', 'NNG', '100'],
 ['Sentence: 1', '.', 'SF', '100'],
 ['Sentence: 1', '정규직', 'NNG', '100'],
 ['Sentence: 1', '으로', 'JKM', '100'],
 ['Sentence: 1', '다니', 'VV', '100'],
 ['Sentence: 1', '기', 'ETN', '100'],
 ['Sentence: 1', '에', 'VV', '100'],
 ['Sentence: 1', 'ㄴ', 'ETD', '100'],
 ['Sentence: 1', '다소', 'MAG', '100'],
 ['Sentence: 1', '피곤', 'NNG', '100'],
 ['Sentence: 1', '하', 'XSV', '100'],
 ['Sentence: 1', 'ㄹ', 'ETD', '100'],
 ['Sentence: 1', '수', 'NNB', '100'],
 ['Sentence: 1', '도', 'JX', '100'],
 ['Sentence: 2', '필요', 'NNG', '100'],
 ['Sentence: 2', '하', 'XSV', '100'],
 ['Sentence: 2', 'ㄴ', 'ETD', '100

In [23]:
text1_sent

[['Sentence: 1', '계약', 'NNG', '100'],
 ['Sentence: 1', '직', 'NNG', '100'],
 ['Sentence: 1', '은', 'JX', '100'],
 ['Sentence: 1', '아무', 'MDT', '100'],
 ['Sentence: 1', '생각', 'NNG', '100'],
 ['Sentence: 1', '없이', 'MAG', '100'],
 ['Sentence: 1', '다니', 'VV', '100'],
 ['Sentence: 1', '기', 'ETN', '100'],
 ['Sentence: 1', '좋', 'VA', '100'],
 ['Sentence: 1', '은', 'ETD', '100'],
 ['Sentence: 1', '곳', 'NNG', '100'],
 ['Sentence: 1', '.', 'SF', '100'],
 ['Sentence: 1', '정규직', 'NNG', '100'],
 ['Sentence: 1', '으로', 'JKM', '100'],
 ['Sentence: 1', '다니', 'VV', '100'],
 ['Sentence: 1', '기', 'ETN', '100'],
 ['Sentence: 1', '에', 'VV', '100'],
 ['Sentence: 1', 'ㄴ', 'ETD', '100'],
 ['Sentence: 1', '다소', 'MAG', '100'],
 ['Sentence: 1', '피곤', 'NNG', '100'],
 ['Sentence: 1', '하', 'XSV', '100'],
 ['Sentence: 1', 'ㄹ', 'ETD', '100'],
 ['Sentence: 1', '수', 'NNB', '100'],
 ['Sentence: 1', '도', 'JX', '100'],
 ['Sentence: 2', '필요', 'NNG', '100'],
 ['Sentence: 2', '하', 'XSV', '100'],
 ['Sentence: 2', 'ㄴ', 'ETD', '100

In [24]:
#text2_del_SW = []
#for i in range(len(text2_sent)):
#    for t in text1_sent : 
#        if ( (t[2] != 'ETD') | (t[2] !='SF') | (t[2] !='SP') | (t[2] !='XSV') 
#            | (t[2][:2] !='JK')   ):
#            text2_del_SW.append(t)

In [25]:
#text2_del_SW

In [26]:
#text2_del_SW = []
#for i in range(len(text2_sent)):
#    for t in text2_sent : 
#        if t[1] != 'ETD':
#            text2_del_SW.append()

In [27]:
#dict_pos

In [28]:
#dict_neg

In [29]:
data = pd.DataFrame(text1_sent) # data를 pandas로 변환

In [30]:
data.columns = ['Sentence #', 'Word', 'POS', 'score']

In [31]:
data.head(10)

,Sentence #,Word,POS,score
0,Sentence: 1,계약,NNG,100
1,Sentence: 1,직,NNG,100
2,Sentence: 1,은,JX,100
3,Sentence: 1,아무,MDT,100
4,Sentence: 1,생각,NNG,100
5,Sentence: 1,없이,MAG,100
6,Sentence: 1,다니,VV,100
7,Sentence: 1,기,ETN,100
8,Sentence: 1,좋,VA,100
9,Sentence: 1,은,ETD,100


In [32]:
data.tail(10)

,Sentence #,Word,POS,score
7483,Sentence: 300,와,JKM,0
7484,Sentence: 300,뒷담,NNG,0
7485,Sentence: 300,화,XSN,0
7486,Sentence: 300,가,JKS,0
7487,Sentence: 300,일상인,NNG,0
7488,Sentence: 300,구성원,NNG,0
7489,Sentence: 300,이,JKS,0
7490,Sentence: 300,모이,VV,0
7491,Sentence: 300,ㄴ,ETD,0
7492,Sentence: 300,집단,NNG,0


In [33]:
data.isnull().sum() # 비어있는 값 확인 결과: 없음

Sentence #    0
Word          0
POS           0
score         0
dtype: int64

In [34]:
words = list(set(data["Word"].values))
n_words = len(words) # 고유한 단어들이 1784개(문장부호 포함)
n_words

1500

In [35]:
# 데이터에서 문장을 원하는 형태로 포매팅하여 추출하는 클래스
# Word, POS -> data를 여러 함수로 씌워서 봄. 
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        # for문 사용법 보기. lambda: 일종의 함수(적용하는), data에 다음과 같은 기능 적용
        agg_func = lambda s: [(w, p, s) for w, p, s in zip(s["Word"].values.tolist(),
                                                                     s["POS"].values.tolist(),
                                                                     s["score"].values.tolist())]
        # sentence 단위로 grouped
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        # 그룹화된 데이터를 다시 sentences로 저장
        self.sentences = [s for s in self.grouped]
        print(self.grouped)
        self.sentences
    
    # get_next: 다음 문장 처리
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [36]:
# 전체 문장 추출
getter = SentenceGetter(data)

Sentence #
Sentence: 1      [(계약, NNG, 100), (직, NNG, 100), (은, JX, 100), ...
Sentence: 10     [(개인, NNG, 0), (에게, JKM, 0), (주어지, VV, 0), (ㄴ,...
Sentence: 100    [(멋있, VA, 100), (어요, EFN, 100), (~~~, SW, 100)...
Sentence: 101    [(지방, NNG, 100), (직, NNG, 100), (절, NNG, 100),...
Sentence: 102    [(수도권, NNG, 100), (에, JKM, 100), (위치, NNG, 100...
                                       ...                        
Sentence: 95     [(여성, NNG, 100), (경력, NNG, 100), (단절, NNG, 100...
Sentence: 96     [(사람, NNG, 100), (마다, JX, 100), (느끼, VV, 100),...
Sentence: 97     [(전시, NNG, 100), (,, SP, 100), (프로그램, NNG, 100...
Sentence: 98     [(서울시, NNG, 100), (산하, NNG, 100), (재단, NNG, 10...
Sentence: 99     [(유, NNG, 100), (네스, NNP, 100), (코, NNG, 100),...
Length: 300, dtype: object


In [37]:
# 문장 하나
sent = getter.get_next()

In [38]:
# 추출한 문장 확인(적용 결과)
print(sent)

[('계약', 'NNG', '100'), ('직', 'NNG', '100'), ('은', 'JX', '100'), ('아무', 'MDT', '100'), ('생각', 'NNG', '100'), ('없이', 'MAG', '100'), ('다니', 'VV', '100'), ('기', 'ETN', '100'), ('좋', 'VA', '100'), ('은', 'ETD', '100'), ('곳', 'NNG', '100'), ('.', 'SF', '100'), ('정규직', 'NNG', '100'), ('으로', 'JKM', '100'), ('다니', 'VV', '100'), ('기', 'ETN', '100'), ('에', 'VV', '100'), ('ㄴ', 'ETD', '100'), ('다소', 'MAG', '100'), ('피곤', 'NNG', '100'), ('하', 'XSV', '100'), ('ㄹ', 'ETD', '100'), ('수', 'NNB', '100'), ('도', 'JX', '100')]


In [39]:
# 전체 문장
sentences = getter.sentences

In [40]:
print(len(sentences)) # 개수 확인

# 문장 하나 확인
print(sentences[58])
print(sentences[:2]) # list의 list 형태로 여러 개 표시. 


# 각각 word, pos, tag, pare로 만들어서 결과 확인

300
[('사고', 'NNG', '0'), ('만', 'JX', '0'), ('안', 'MAG', '0'), ('치', 'VV', '0'), ('면', 'ECE', '0'), ('안', 'MAG', '0'), ('짤리', 'VV', '0'), ('는', 'ETD', '0'), ('회사', 'NNG', '0'), ('일', 'NNG', '0'), ('안해', 'NNG', '0'), ('도', 'JX', '0'), ('안', 'MAG', '0'), ('짤', 'VV', '0'), ('려요', 'ECD', '0'), ('프리', 'NNG', '0'), ('라이더', 'NNG', '0'), ('되', 'VV', '0'), ('고', 'ECE', '0'), ('싶', 'VXA', '0'), ('은', 'ETD', '0'), ('사람', 'NNG', '0'), ('다', 'MAG', '0'), ('모이', 'VV', '0'), ('어', 'ECS', '0'), ('라', 'NNG', '0'), ('~', 'SO', '0')]
[[('계약', 'NNG', '100'), ('직', 'NNG', '100'), ('은', 'JX', '100'), ('아무', 'MDT', '100'), ('생각', 'NNG', '100'), ('없이', 'MAG', '100'), ('다니', 'VV', '100'), ('기', 'ETN', '100'), ('좋', 'VA', '100'), ('은', 'ETD', '100'), ('곳', 'NNG', '100'), ('.', 'SF', '100'), ('정규직', 'NNG', '100'), ('으로', 'JKM', '100'), ('다니', 'VV', '100'), ('기', 'ETN', '100'), ('에', 'VV', '100'), ('ㄴ', 'ETD', '100'), ('다소', 'MAG', '100'), ('피곤', 'NNG', '100'), ('하', 'XSV', '100'), ('ㄹ', 'ETD', '100'), ('수', 'NNB'

In [41]:
# 단어를 features화
# "feat의 이름" : "value"
# 하나의 단어에서 다양한 feature(단어, POS tag, lables 등) 확인을 위해 문장 전체를 넣어줌. 

def word2features(sent, i):
    word = sent[i][0]   # index0의 단어를 가져옴
    postag = sent[i][1] # index1의 단어를 가져옴
    score = sent[i][2]

# postag 유지
# postag2 = kkma.pos(word)
# 
    features = {
        'bias': 1.0,
        'postag': postag,
        'postag[:2]': postag[:2],
        'dic_text_pos' : True if word in dic_text_pos1 else False,
        'dic_text_neg' : True if word in dic_text_neg1 else False,
#        'dict_pos' : True if word in dict_pos else False,
#        'dict_neg' : True if word in dict_neg else False,
        'word.len' : len(word)
        
# 특징!!!! 
# 
#'dict_time': word.lower() in dict_time # -> true or false로
# character 개수 
# 'word.len' : len(word)
    }
    
    # 직전의 단어에 대해서도 똑같이 적용
    if i > 0:
        word1 = sent[i-1][0] # i-1: 직전의 단어
        postag1 = sent[i-1][1]
        # update: 기존에서 추가! (하여 새로운 dictionary를 만드는 결과)
        features.update({
            '-1:postag': postag,
            'dic_text_pos' : True if word in dic_text_pos1 else False,
            'dic_text_neg' : True if word in dic_text_neg1 else False,
#            'dict_pos' : True if word in dict_pos else False,
#            'dict_neg' : True if word in dict_neg else False,
            '-1:word.len' : len(word)
        })
    #else:
    #    features['BOS'] = True # BOS: Beginning Of Sentence

    # 다음 단어에 대해서도 똑같이 처리
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag,
            'dic_text_pos' : True if word in dic_text_pos1 else False,
            'dic_text_neg' : True if word in dic_text_neg1 else False,
#            'dict_pos' : True if word in dict_pos else False,
#            'dict_neg' : True if word in dict_neg else False,
            '+1:word.len' : len(word)
        })
    
    #else:
    #    features['EOS'] = True  # EOS: end of sentence
        
    # bi-그램에 대해서도 똑같이 처리
    
    return features

# sentence 전체에 적용하기 위한 함수
def sent2features(sent):
    for i in range(len(sent)):
        print(word2features(sent, i))
    return [word2features(sent, i) for i in range(len(sent))] # 0~i(index의 수)까지 적용 -> sentence에 적용
# == features = [word2features(sent, i) for i in range(len(sent))]
# return features

def sent2score(sent): # sent를 읽어서 for문을 돌림. label만 갖고 옴
    return [score for token, postag, score in sent]
# 지금은 3개가 tuple로 있음.

def sent2tokens(sent):
    return [token for token, postag, score in sent]

In [42]:
sent2score(sent)

['100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100']

In [43]:
print(sent2features(sent))

{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': True, 'dic_text_neg': True, 'word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'NNG', '-1:word.len': 1, '+1:postag': 'NNG', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'JX', 'postag[:2]': 'JX', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'JX', '-1:word.len': 1, '+1:postag': 'JX', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'MDT', 'postag[:2]': 'MD', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 2, '-1:postag': 'MDT', '-1:word.len': 2, '+1:postag': 'MDT', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': True, 'dic_text_neg': True, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'MAG', 'postag[:2]': 'MA', 'dic_text_pos': True, 'dic_text_neg': True, 'word.len'

In [44]:
X = [sent2features(s) for s in sentences] # sentence마다 feature로 바뀌고
y = [sent2score(s) for s in sentences] # sentence마다 score로 바뀜

{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': True, 'dic_text_neg': True, 'word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'NNG', '-1:word.len': 1, '+1:postag': 'NNG', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'JX', 'postag[:2]': 'JX', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'JX', '-1:word.len': 1, '+1:postag': 'JX', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'MDT', 'postag[:2]': 'MD', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 2, '-1:postag': 'MDT', '-1:word.len': 2, '+1:postag': 'MDT', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': True, 'dic_text_neg': True, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'MAG', 'postag[:2]': 'MA', 'dic_text_pos': True, 'dic_text_neg': True, 'word.len'

In [45]:
y

[['100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['100', '100', '100', '100', '100', '100', '100', '100', '100', '100'],
 ['100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100'],
 ['100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100'],
 ['100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100'

In [46]:
print(len(X))
print(len(y))
print(len(sentences))

300
300
300


In [47]:
#10번째 sentence의 정보 
print(len(X[10])) 
print(len(y[10]))
print(sentences[10])
print(y[10])
print(X[10])

22
22
[('대구', 'NNG', '100'), ('가', 'JKS', '100'), ('연고', 'NNG', '100'), ('지인', 'NNG', '100'), ('사람', 'NNG', '100'), ('들', 'XSN', '100'), ('은', 'JX', '100'), ('평생', 'NNG', '100'), ('대구', 'NNG', '100'), ('에서', 'JKM', '100'), ('살', 'VV', '100'), ('ㄹ', 'ETD', '100'), ('수', 'NNB', '100'), ('있', 'VV', '100'), ('다는', 'ETD', '100'), ('것', 'NNB', '100'), ('이', 'JKS', '100'), ('장점', 'NNG', '100'), ('이', 'VCP', '100'), ('ㄴ', 'ETD', '100'), ('직렬', 'NNG', '100'), ('..', 'SW', '100')]
['100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100']
[{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}, {'bias': 1.0, 'postag': 'JKS', 'postag[:2]': 'JK', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'JKS', '-1:word.len': 1, '+1:postag': 'JKS', '+1:word.len': 1}, {'bias': 1.0, 'postag': 'NNG

In [48]:
# 문장 하나 확인
# 각 단어의 features가 dictionary로 표현되고 이를 요소로 하는 리스트
X[0]
# 문장은 단어로 이루어져 있고, 각 단어의 features 확인 가능
# 첫 번째 단어는 앞 단어에 대한 정보는 없고 BOS = true

[{'bias': 1.0,
  'postag': 'NNG',
  'postag[:2]': 'NN',
  'dic_text_pos': True,
  'dic_text_neg': True,
  'word.len': 2,
  '+1:postag': 'NNG',
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'NNG',
  'postag[:2]': 'NN',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 1,
  '-1:postag': 'NNG',
  '-1:word.len': 1,
  '+1:postag': 'NNG',
  '+1:word.len': 1},
 {'bias': 1.0,
  'postag': 'JX',
  'postag[:2]': 'JX',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 1,
  '-1:postag': 'JX',
  '-1:word.len': 1,
  '+1:postag': 'JX',
  '+1:word.len': 1},
 {'bias': 1.0,
  'postag': 'MDT',
  'postag[:2]': 'MD',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 2,
  '-1:postag': 'MDT',
  '-1:word.len': 2,
  '+1:postag': 'MDT',
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'NNG',
  'postag[:2]': 'NN',
  'dic_text_pos': True,
  'dic_text_neg': True,
  'word.len': 2,
  '-1:postag': 'NNG',
  '-1:word.len': 2,
  '+1:postag': 'NNG',
  '+1:word.len': 2},
 {'bias':

In [49]:
print(len(X), len(X)*0.7) #train-test split (70%, 30%)
print(type(X))

300 210.0
<class 'list'>


In [50]:
crf = CRF(algorithm='lbfgs',
          c1=0.1, #c1의 가중치를 조절하여 feature의 영향력 조절
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [51]:

# Training
crf.fit(X[:300], y[:300])

C:\ProgramData\Anaconda3\envs\tm\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [52]:
# cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5) 

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [53]:
# prediction
y_test_pred = crf.predict(X[300:])

In [54]:
# report
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

# data + deep learning 적용 시 늘어날 가능성은 있음. 

              precision    recall  f1-score   support

           0       0.49      0.51      0.50      3928
         100       0.43      0.41      0.42      3565

    accuracy                           0.46      7493
   macro avg       0.46      0.46      0.46      7493
weighted avg       0.46      0.46      0.46      7493



C:\ProgramData\Anaconda3\envs\tm\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [55]:
# 태그 간의 transition(전이) probabilities, 태그 별 예측에 중요한 features
# eli5: weight visualization
eli5.show_weights(crf, top=30) # -> 중요한 것만 확인

In [56]:
# 시각화 하려고 했지만 실패한 흔적

explainer = LimeTextExplainer(class_names='CRF_NER')

NameError: name 'LimeTextExplainer' is not defined

In [ ]:
exp1 = explainer.explain_instance(text1[0], pred, num_features=5)